<a href="https://colab.research.google.com/github/Artemis-2/IRE_Major_Project/blob/main/PLS_WORK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
# from torch._C import *
import torch.nn as nn
import random
import json
import os
import string
import torch.nn.functional as F
import tensorflow_datasets.public_api as tfds
from gensim.models import FastText
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# data_zip = drive.CreateFile({'id':'1J3mucMFTWrgAYa3LuBZoLRR3CzzYD3fa'}) 
# data_zip.GetContentFile('ProjectData')

In [ ]:
# import zipfile
# zip_ref = zipfile.ZipFile("/content/drive/My Drive/data_zip.zip", 'r')
# zip_ref.extractall("/tmp")
# zip_ref.close()

In [ ]:
file = open('/content/drive/My Drive/BiGPatentSample.txt',mode='r')
 
# read all lines at once
my_data = file.read()
 
# close the file
file.close()

# file=open("/content/drive/My Drive/train_000 (1).bin", "rb")

# my_data=file.read()
# file.close()




In [ ]:
stop_file = open("/content/drive/My Drive/NLTK's list of english stopwords",mode='r')
stop_words = stop_file.read()
stop_file.close()
stop_words = stop_words.split('\n')
print(stop_words)





['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [ ]:
from gensim.models import word2vec
from gensim.models import KeyedVectors
wv_from_bin = KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin',binary=True) 

# model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin', binary=True)
# model.save_word2vec_format('content/drive/My Drive/GoogleNews-vectors-negative300.bin', binary=False)

import gensim
# model_ted = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)
weights = wv_from_bin.wv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:

 
dataset = my_data.split('}')

x = []
y = []
for itm in dataset :
  abst =  itm.split("\"abstract\":",1)
  if len(abst)!=1:
    abst = abst[1].split("\"application_number\":")[0]
    y.append(abst)
  full_txt = itm.split("\"description\":")
  if len(full_txt)!=1:
    x.append(full_txt[1])
  # print(full_txt)

In [ ]:
# print(x[10])
# print(y[10])
# print(wv_from_bin['king'].shape)
wv_from_bin['<START>'] = torch.zeros(wv_from_bin['king'].shape)
wv_from_bin['<END>'] = torch.zeros(wv_from_bin['king'].shape)
wv_from_bin[''] = torch.zeros(wv_from_bin['king'].shape)

In [ ]:
def clean(text):
  text = str(text)
  text = text.lower()
  text = re.sub(r'\'s',r'\tis',text)
  text = re.sub(r'\'ll',r'\twill',text)
  text = re.sub(r'\'m',r'\tam',text)
  text = re.sub(r'\'re',r'\tare',text)
  text = re.sub(r'\'d',r'\twould',text)
  text = re.sub(r'n\'t',r'\tnot',text)
  # print("1***",text)
  for itm in stop_words:
    text = re.sub(' '+itm +' ',' ',text)
    text = re.sub('"'+ itm + ' ',' ', text)
  
  # print("2***", text)
  text = re.sub('fig[0-9]+',' ',text)
  text = re.sub('[^a-zA-Z0-9]',' ',text) 
  text = re.sub('\w{1,}[0-9]+', ' ', text)
  text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
  text = re.sub('^a$','', text)
  return text

In [ ]:
cleaned_source = list(map(clean,x))
cleaned_summary = list(map(clean,y))

for i in range(len(cleaned_summary)):
    cleaned_summary[i] = "<START> " + cleaned_summary[i] + " <END>"
    
# print(cleaned_source[11])
# print(cleaned_summary[11])



In [ ]:
min_source_length = 10000000000000000
max_source_length = 0
min_target_length = 10000000000000000
max_target_length = 0

for i in range(len(x)):
  min_source_length = min(min_source_length,len(cleaned_source[i].split()))
  min_target_length = min(min_target_length,len(cleaned_summary[i].split()))
  max_source_length = max(max_source_length,len(cleaned_source[i].split()))
  max_target_length = max(max_target_length,len(cleaned_summary[i].split()))

print("Minimum source length is:  ",min_source_length)
print("Minimum target length is:  ",min_target_length)
print("Maximum source length is:  ",max_source_length)
print("Maximum target length is:  ",max_target_length)

Minimum source length is:   387
Minimum target length is:   14
Maximum source length is:   4463
Maximum target length is:   118


In [ ]:
new_source = []
new_summary = []

for i in range(len(cleaned_source)):
  # if len(cleaned_source[i].split()) <= 50 and len(cleaned_summary[i].split()) <= 15 :
  #   new_source.append(cleaned_source[i])
  #   new_summary.append(cleaned_summary[i])
  new_source.append(cleaned_source[i])
  new_summary.append(cleaned_summary[i])
# print(new_source)

max_source_length = 7706
max_summary_length = 214


new_source = new_source[:1000]
new_summary = new_summary[:1000]

# print(len(new_source))
# print(len(new_summary))


In [ ]:
sentences = new_source + new_summary
sent_ted = []
# print(sentences)
for sent in sentences:
  sent_ted_child = sent.split()
  sent_ted.append(sent_ted_child)

# print(sent_ted[0])

In [ ]:
## Approach 1
# class TheModelClass(nn.Module):
#     def __init__(self):
#         super(TheModelClass, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)
# model = TheModelClass()
# model_ted = torch.load('/content/drive/My Drive/see_etal_weights_with_coverage.th')
# print(model_ted)
# full_model = model_ted
## Approach 2
# import pickle
# model_ted = pickle.load(open('/content/drive/My Drive/128_emb.pkl', 'rb'))
# weights = model_ted.wv
# print(model_ted.wv.most_similar("milk"))

# print(model_ted['parameters'])
# model_ted.eval()


In [ ]:

from collections import OrderedDict
word2Index_enc = {}
word2Index_dec = {}
word2Index_dec_big = {}

ind2Word_enc = {}
ind2Word_dec = {}
ind2Word_dec_big = {}

word2PsuInd_dec = {}
psuInd2Word_dec = {}

encoder_paragraph = list(set((' '.join(new_source)).split()))

decoder_paragraph_list = list((' '.join(new_summary)).split())
decoder_dict = OrderedDict()
for word in decoder_paragraph_list:
  try:
    if word!='' and word!=' ':
      decoder_dict[word] = decoder_dict[word] + 1
  except:
    if word!='' and word!=' ':
      decoder_dict[word] = 1
# print(decoder_dict)
ind2Word_enc[0] = '<UNK>'
ind2Word_dec[0] = '<UNK>'
word2Index_enc['<UNK>'] = 0
word2Index_dec['<UNK>'] = 0
ind2Word_dec_big[0] = '<UNK>'
word2Index_dec_big['<UNK>'] = 0
word2PsuInd_dec['<UNK>'] = 0
psuInd2Word_dec[0] = '<UNK>'

dec_index = 1
for (decoder_dict_word, decoder_dict_number) in decoder_dict.items():
  word2Index_dec_big[decoder_dict_word] = dec_index
  ind2Word_dec_big[dec_index] = decoder_dict_word
  if decoder_dict_number >= 3 :
    word2Index_dec[decoder_dict_word] = dec_index
    ind2Word_dec[dec_index] = decoder_dict_word
    psuedo_index = len(word2PsuInd_dec.keys())
    word2PsuInd_dec[decoder_dict_word] = psuedo_index
    psuInd2Word_dec[psuedo_index] = decoder_dict_word
  dec_index+=1

enc_index = 1
for index,word in enumerate(encoder_paragraph):
  if word != ' ' and word!= '':
    word2Index_enc[word] = enc_index
    ind2Word_enc[enc_index] = word 
    enc_index+=1

In [ ]:

# encoder_input = [[word2Index_enc[word] for word in sentence.split() if word in word2Index_enc.keys()] for sentence in new_source ]
# decoder_input = [[word2Index_dec_big[word] for word in sentence.split() if word in word2Index_dec_big.keys()] for sentence in new_summary ]


encoder_input = [[word2Index_enc[word] for word in sentence.split() if word in wv_from_bin.vocab] for sentence in new_source ]
decoder_input = [[word2Index_dec_big[word] for word in sentence.split() if word in wv_from_bin.vocab] for sentence in new_summary ]


In [ ]:
encoder_tensor = [torch.tensor(li,dtype=torch.long,device=device).view(-1, 1) for li in encoder_input]
decoder_tensor = [torch.tensor(li,dtype=torch.long,device=device).view(-1, 1) for li in decoder_input]

In [ ]:
class Encoder(nn.Module):
  def __init__(self,input_vocab_size,hidden_size,num_layers=1,bidirectional=False):
    super(Encoder,self).__init__()
    self.bidirectional = bidirectional
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.input_vocab_size = input_vocab_size
    self.gru_layer = nn.GRU(input_size = self.hidden_size,hidden_size = self.hidden_size,num_layers = self.num_layers)

  def forward(self,input_,prev_hidden_state):
    input_word = ind2Word_enc[input_.data.tolist()[0]]
    embedded_outputs = torch.tensor(weights[input_word], device = device).view(1,1,-1)
    output,prev_hidden_state = self.gru_layer(embedded_outputs,prev_hidden_state)  #output is batch_size times hidden_size
    return output,prev_hidden_state

  def init_hidden(self):
    return torch.zeros(1,1,self.hidden_size,device=device)

In [ ]:
class AttentionDecoder(nn.Module):
  def __init__(self,output_vocab_size,hidden_size,max_length_encoder,dropout_value,num_layers=1):
      super(AttentionDecoder,self).__init__()
      self.hidden_size = hidden_size
      self.num_layers = num_layers
      self.output_vocab_size = output_vocab_size
      self.dropout_p = dropout_value
      self.max_length_encoder = max_length_encoder
      self.embedding_layer = nn.Embedding(self.output_vocab_size,self.hidden_size)
      self.attention_layer = nn.Linear(self.hidden_size*2,self.max_length_encoder)
      self.attention_combine = nn.Linear(self.hidden_size*2,self.hidden_size)

      self.s_layer = nn.Linear(self.hidden_size, 1)
      self.x_layer = nn.Linear(self.hidden_size, 1)
      self.context_layer = nn.Linear(self.hidden_size, 1)
      self.linear_pgen = nn.Linear(3, 1)

      self.gru_layer = nn.GRU(self.hidden_size,self.hidden_size)
      self.output_layer = nn.Linear(self.hidden_size,self.output_vocab_size)
      self.dropout_layer = nn.Dropout(self.dropout_p)    
  def forward(self,input_,prev_hidden_state,encoder_output,prev_unk_word):
      input_word = ind2Word_dec_big[input_.data.tolist()[0]]
      if input_word == '<UNK>':
        embedded_outputs = torch.tensor(weights[prev_unk_word], device = device).view(1,1,-1)
      else:
        embedded_outputs = torch.tensor(weights[input_word], device = device).view(1,1,-1)
        
      embeddings_dropout = self.dropout_layer(embedded_outputs)
      attention_layer_output = self.attention_layer(torch.cat((embeddings_dropout[0],prev_hidden_state[0]),1))
      attention_weights = nn.functional.softmax(attention_layer_output,dim=1)
      attention_applied = torch.bmm(attention_weights.unsqueeze(0),encoder_output.unsqueeze(0))
      attention_combine_logits = self.attention_combine(torch.cat((embeddings_dropout[0],attention_applied[0]),1)).unsqueeze(0)  #since gru requires a batch dimension
      attention_combine_relu = nn.functional.relu(attention_combine_logits)

      s_output = self.s_layer(prev_hidden_state[0])
      x_output = self.x_layer(embeddings_dropout[0])
      context = torch.flatten(attention_applied)
      context_weights = self.context_layer(attention_applied)
      sx = torch.cat((s_output[0],x_output[0]),0)
      sxc = torch.cat((sx,context_weights[0][0]),0)
      linear_pgen = self.linear_pgen(sxc)
      m = nn.Sigmoid()
      pgen = m(linear_pgen)

      output,hidden = self.gru_layer(attention_combine_relu,prev_hidden_state)
      output_logits = self.output_layer(output)
      output_softmax = nn.functional.log_softmax(output_logits[0],dim=1)
      return output_softmax,hidden,attention_weights,pgen
  def init_hidden(self):
    return torch.zeros(1,1,self.hidden_size,device=device)

In [ ]:

teacher_forcing_ratio = 0.5

def train(encoder, decoder, input_tensor, target_tensor, encoder_optimizer, decoder_optimizer, criterion, max_length, iters):
  # print("HERE 1")
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()
  # print("HERE2")

  prev_unk_word = ''
  # prev_unk_word = '<START>'

  encoder_hidden = encoder.init_hidden()

  encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)

  input_length = input_tensor.size(0)
  output_length = target_tensor.size(0)

  loss = 0
  # print("HERE2.5")
  for encoder_index in range(0, input_length):
    encoder_output,encoder_hidden = encoder(input_tensor[encoder_index], encoder_hidden)
    encoder_outputs[encoder_index] = encoder_output[0,0]
  # print("HERE3")
  decoder_input = torch.tensor([word2Index_dec['<START>']],device=device)   
  decoder_hidden = encoder_hidden
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
  # print("HERE4")
  extended_vocab = psuInd2Word_dec.copy()
  reverse_extended_vocab = word2PsuInd_dec.copy()
  duplicate_words = {}
  extend_key = len(word2Index_dec.keys())
  input_list = input_tensor.tolist()
  #print(input_list)
  i =0
  for input_word in input_list:
    # print(input_word)
    if ind2Word_enc[input_word[0]] in word2Index_dec.keys():
      duplicate_words[i] = word2PsuInd_dec[ind2Word_enc[input_word[0]]]
    else:
      extended_vocab[extend_key] = ind2Word_enc[input_word[0]]
      reverse_extended_vocab[ind2Word_enc[input_word[0]]] = extend_key
      extend_key += 1
    i = i+1
  # print("HEERE6")  
  if use_teacher_forcing:
    # print("This branch")
    for decoder_index in range(output_length):
      # decoder_output,decoder_hidden,decoder_attention,pgen = decoder(decoder_input,decoder_hidden,encoder_outputs,prev_unk_word)
      decoder_output,decoder_hidden,decoder_attention,pgen = decoder(decoder_input,decoder_hidden,encoder_outputs,prev_unk_word)
      # print("sadsd")
      P_over_extended_vocab = torch.exp(decoder_output)*pgen.expand_as(torch.exp(decoder_output))
      # print("HERE6.5")
      decoder_attention = decoder_attention.squeeze(0)[0:input_length].unsqueeze(0)
      p_duplicate_list = torch.zeros([input_length, P_over_extended_vocab.size(1)], device=device)
      p_duplicate_list = p_duplicate_list.tolist()
      for (duplicate_word_key,duplicate_word_value) in duplicate_words.items():
        p_duplicate_list[duplicate_word_key][duplicate_word_value] = 1
      p_duplicate = torch.tensor(p_duplicate_list, dtype=torch.float, device=device)
      p_diag = torch.mm(decoder_attention, p_duplicate)
      p_diag = p_diag*(torch.tensor([1], device=device).sub(pgen)).expand_as(p_diag)
      p_add_diag = torch.diag(p_diag.squeeze(0),diagonal=0)
      P_over_extended_vocab = torch.mm(P_over_extended_vocab,p_add_diag).add(P_over_extended_vocab)
      # print("HERE7")
      for i in range(input_length):
        if not (1 in p_duplicate_list[i]):
          P_over_extended_vocab = torch.cat((P_over_extended_vocab[0], torch.mm(decoder_attention.squeeze(0)[i].unsqueeze(0).unsqueeze(0), torch.tensor([1], device=device).sub(pgen).unsqueeze(0)).squeeze(0)),0).unsqueeze(0)

      try:
        loss += -torch.log(P_over_extended_vocab[0][ reverse_extended_vocab[ ind2Word_dec_big[ target_tensor[decoder_index].item() ] ] ] + 1e-12)
        loss.backward(retain_graph=True)
      except KeyError:
        loss += torch.tensor(0,dtype=torch.float,device=device)
      decoder_input = target_tensor[decoder_index]
  else:

    for decoder_index in range(output_length):
      decoder_output,decoder_hidden,decoder_attention,pgen = decoder(decoder_input,decoder_hidden,encoder_outputs,prev_unk_word) 
      P_over_extended_vocab = torch.exp(decoder_output)*pgen.expand_as(torch.exp(decoder_output))

      decoder_attention = decoder_attention.squeeze(0)[0:input_length].unsqueeze(0)
      p_duplicate_list = torch.zeros([input_length, P_over_extended_vocab.size(1)], device=device)
      p_duplicate_list = p_duplicate_list.tolist()
      for (duplicate_word_key,duplicate_word_value) in duplicate_words.items():
        p_duplicate_list[duplicate_word_key][duplicate_word_value] = 1
      p_duplicate = torch.tensor(p_duplicate_list, dtype=torch.float, device=device)
      p_diag = torch.mm(decoder_attention, p_duplicate)
      p_diag = p_diag*(torch.tensor([1], device=device).sub(pgen)).expand_as(p_diag)
      p_add_diag = torch.diag(p_diag.squeeze(0),diagonal=0)
      P_over_extended_vocab = torch.mm(P_over_extended_vocab,p_add_diag).add(P_over_extended_vocab)

      for i in range(input_length):
        if not (1 in p_duplicate_list[i]):
          P_over_extended_vocab = torch.cat((P_over_extended_vocab[0], torch.mm(decoder_attention.squeeze(0)[i].unsqueeze(0).unsqueeze(0), torch.tensor([1], device=device).sub(pgen).unsqueeze(0)).squeeze(0)),0).unsqueeze(0)

      try:
        loss += -torch.log(P_over_extended_vocab[0][ reverse_extended_vocab[ ind2Word_dec_big[ target_tensor[decoder_index].item() ] ] ] + 1e-12)
        loss.backward(retain_graph=True)
      except KeyError:
        loss += torch.tensor(0,dtype=torch.float,device=device)
      idx = torch.topk(P_over_extended_vocab, k=1, dim=1)[1]
      if idx.item() < len(word2Index_dec.keys()):   
        decoder_input = torch.tensor([idx.item()],dtype=torch.long,device=device)
      elif idx.item() >= len(word2Index_dec.keys()):
        prev_unk_word = extended_vocab[idx.item()]
        decoder_input = torch.tensor([0],dtype=torch.long,device=device)
      elif (decoder_input.item() == word2Index_dec['<END>']):
        break

  if iters > 20000:
    torch.nn.utils.clip_grad_norm_(rnn_encoder.parameters(),0.4)
    torch.nn.utils.clip_grad_norm_(rnn_decoder.parameters(),0.4)

  encoder_optimizer.step()
  decoder_optimizer.step()

  return loss.item()/output_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    if percent != 0:
      es = s / (percent)
      rs = es - s
      return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
    else:
      return 0

In [ ]:
arr = np.arange(len(encoder_tensor))
np.random.shuffle(arr)
len(arr)


56

In [ ]:
loss_graph = {}

def train_Iters(encoder,decoder,n_iters,print_every=50, plot_every=100,learning_rate = 0.03):
  # start = time.time()
  plot_losses = []
  print_loss_total = 0  # Reset every print_every
  plot_loss_total = 0

  encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = torch.optim.SGD(decoder.parameters(), lr=learning_rate)
  training_pairs = [random.choice(pairs) for i in range(n_iters)]
  # print("HI1")
  criterion = nn.NLLLoss()
  for iters in range(n_iters):
    training_pair = training_pairs[iters - 1]
    input_tensor = training_pair[0]
    target_tensor = training_pair[1]
    # print("hi2")
    input_tensor = torch.tensor(input_tensor, dtype=torch.long, device = device).view(-1, 1)
    target_tensor = torch.tensor(target_tensor, dtype=torch.long, device = device).view(-1, 1)
    # print("hi3")
    loss = train(encoder,decoder,input_tensor,target_tensor,encoder_optimizer,decoder_optimizer,criterion,max_source_length, iters)
    print_loss_total += loss
    plot_loss_total += loss

    if iters % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print('%s %d%%) %.4f' % (iters, iters / len(arr) * 100, print_loss_avg))

        if iters > 0:
          loss_graph[iters] = print_loss_avg

    if iters % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0


In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
pairs = []
for enc,dec in zip(encoder_input,decoder_input):
    pairs.append([enc,dec])

In [ ]:
hidden_size = 300
rnn_encoder = Encoder(len(word2Index_enc.keys()),hidden_size).to(device=device)
# rnn_encoder = Encoder(len(wv_from_bin.vocab),hidden_size).to(device=device)
rnn_decoder = AttentionDecoder(len(word2Index_dec.keys()),hidden_size,max_source_length,0.2).to(device=device)
# rnn_decoder = AttentionDecoder(len(wv_from_bin.vocab),hidden_size,max_source_length,0.2).to(device=device)


train_Iters(rnn_encoder,rnn_decoder,100000)

In [ ]:
import matplotlib.pyplot as plt

iters = list(loss_graph.keys())
loss_val = list(loss_graph.values())
plt.plot(iters, loss_val)
plt.ylim(0,8) 
plt.xlim(0,77350)
plt.xlabel('Iterations') 
plt.ylabel('Loss')  
plt.show()

In [ ]:

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        pair1 = torch.tensor(pair[0],dtype=torch.long,device=device)
        pair2 = pair[1]
        output_words, attentions = evaluate(encoder, decoder, pair1)
        output_sentence = ' '.join(output_words)
        output_list = [ind2Word_dec_big[word] for word in pair2]
        output_list = ' '.join(output_list)
        input_sentence = [ind2Word_enc[element.item()] for element in pair1.flatten()]
        input_sentence = ' '.join(input_sentence)
        print("Sentence is  ",input_sentence)
        print('<',output_sentence)
        print('=',output_list)


In [ ]:
        def evaluate(encoder, decoder, encoder_tensor, max_length=max_source_length):
            with torch.no_grad():
                input_tensor = encoder_tensor
                input_length = input_tensor.size(0)
                encoder_hidden = encoder.init_hidden()

                prev_unk_word = ''

                encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

                for ei in range(input_length):
                    encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0),
                                                            encoder_hidden)
                    encoder_outputs[ei] += encoder_output[0, 0]

                extended_vocab = psuInd2Word_dec.copy()
                duplicate_words = {}
                extend_key = len(word2Index_dec.keys())
                input_list = input_tensor.tolist()
                i =0
                for input_word in input_list:
                  if ind2Word_enc[input_word] in word2Index_dec.keys():
                    duplicate_words[i] = word2PsuInd_dec[ind2Word_enc[input_word]]
                  else:
                    extended_vocab[extend_key] = ind2Word_enc[input_word]
                    extend_key += 1
                  i = i+1

                decoder_input = torch.tensor([word2Index_dec['<START>']], device=device)  # SOS

                decoder_hidden = encoder_hidden

                decoded_words = []
                decoder_attentions = torch.zeros(max_length, max_length)

                for di in range(max_length):
                    decoder_output, decoder_hidden, decoder_attention,pgen = decoder(
                        decoder_input, decoder_hidden, encoder_outputs, prev_unk_word)
                    decoder_attentions[di] = decoder_attention.data

                    P_over_extended_vocab = torch.exp(decoder_output)*pgen.expand_as(torch.exp(decoder_output))

                    decoder_attention = decoder_attention.squeeze(0)[0:input_length].unsqueeze(0)
                    p_duplicate_list = torch.zeros([input_length, P_over_extended_vocab.size(1)], device=device)
                    p_duplicate_list = p_duplicate_list.tolist()
                    for (duplicate_word_key,duplicate_word_value) in duplicate_words.items():
                      p_duplicate_list[duplicate_word_key][duplicate_word_value] = 1
                    p_duplicate = torch.tensor(p_duplicate_list, dtype=torch.float, device=device)
                    p_diag = torch.mm(decoder_attention, p_duplicate)
                    p_diag = p_diag*(torch.tensor([1], device=device).sub(pgen)).expand_as(p_diag)
                    p_add_diag = torch.diag(p_diag.squeeze(0),diagonal=0)
                    P_over_extended_vocab = torch.mm(P_over_extended_vocab,p_add_diag).add(P_over_extended_vocab)

                    for i in range(input_length):
                      if not (1 in p_duplicate_list[i]):
                        P_over_extended_vocab = torch.cat((P_over_extended_vocab[0], torch.mm(decoder_attention.squeeze(0)[i].unsqueeze(0).unsqueeze(0), torch.tensor([1], device=device).sub(pgen).unsqueeze(0)).squeeze(0)),0).unsqueeze(0)

                    idx = torch.topk(P_over_extended_vocab, k=1, dim=1)[1]
                    if idx.item() < len(word2Index_dec.keys()):   
                      decoder_input = torch.tensor([idx.item()],dtype=torch.long,device=device)
                      decoded_words.append(extended_vocab[idx.item()])
                    elif idx.item() >= len(word2Index_dec.keys()):
                      decoder_input = torch.tensor([0],dtype=torch.long,device=device)
                      prev_unk_word = extended_vocab[idx.item()]
                      decoded_words.append(extended_vocab[idx.item()])
                    if idx.item() == word2Index_dec['<END>']:
                      decoded_words.append('<END>')
                      break

                return decoded_words, decoder_attentions[:di + 1]

In [ ]:
evaluateRandomly(rnn_encoder, rnn_decoder)